In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [4]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:00


In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 130.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 61.1 MB/s eta 0:00:00


In [6]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.7 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=603f541b283a593be87817bace65611f7531c6fd1acfe93e93153f0399d590e8
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


# Requirements

In [15]:
import torch

from torch.optim import Adam
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE
from torch_geometric.data import Data

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import Node2VecConfig, Node2VecOptimizerConfig, SimpleClassifierConfig
from src.features import get_node2vec_pair_embedd_for_training_data, get_associations, get_gae_pair_embedd_for_training_data
from src.config import SimpleClassifierConfig, GraphAutoEncoderConfig, KGEConfig
from src.features import get_relations, get_entities, get_associations, get_homogeneous_graph, get_kge_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory, MDAKnowledgeGraphEmbedding
from src.utils import train_test_sampler, prj_logger
from src.optimization import KGETrainer
from torch_geometric.nn import GCNConv

In [74]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
from io import BytesIO
from PIL import Image
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# Data

In [105]:
associations = get_associations()
y = associations['increased'].tolist()

In [106]:
data = Data()
data.num_nodes = torch.tensor(get_entities()['id'].shape[0]).to(device)
data.edge_index = torch.tensor([get_relations()['head'].tolist(), get_relations()['tail'].tolist()]).to(device)
data.edge_type = torch.tensor(get_relations()['relation'].tolist()).to(device)

In [107]:
disease_list = get_associations()['disease'].tolist()
microbe_list = get_associations()['microbe'].tolist()
node_list = get_homogeneous_graph(device).x.squeeze().detach().tolist()

2024-02-11 18:23:11,860 [INFO] Calling get_homogeneous_graph
2024-02-11 18:23:12,345 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])


# ComplEx

In [108]:
kge_optimizer_config = OptimizerConfig()
kge_optimizer_config.optimizer = torch.optim.Adam
kge_optimizer_config.lr = 0.01
kge_optimizer_config.batch_size = 1000
kge_optimizer_config.n_epoch = 1
kge_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
kge_optimizer_config.device = device

In [109]:
kge_model_config = KGEConfig()
kge_model_config.kge = ComplEx
kge_model_config.hidden_channels = 32
kge_model_config.num_relations = data.num_edge_types
kge_model_config.num_nodes = data.num_nodes

In [110]:
kge_model = MDAKnowledgeGraphEmbedding(kge_model_config)

trainer = KGETrainer()

2024-02-11 18:23:13,326 [INFO] Initialing MDATransE with model_config {'model_name': None}


In [111]:
iteration = 100
time_embedds = []

for i in range(iteration):
    trainer.train(model=kge_model, data=data, config=kge_optimizer_config)

    all_embedd = kge_model.predict(node_list=node_list)
    d_embedd = all_embedd[disease_list]
    m_embedd = all_embedd[microbe_list]
    md_embed = torch.cat((d_embedd, m_embedd), 1).detach().to(kge_optimizer_config.device)

    tsne = TSNE(n_components=2, random_state=42)
    time_embedds.append(tsne.fit_transform(md_embed)[:, :2])

2024-02-11 18:23:22,402 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-11 18:23:22,404 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-11 18:23:22,406 [INFO] moving model to cpu
2024-02-11 18:23:38,747 [INFO] Epoch: 001, Loss: 0.4251
2024-02-11 18:23:38,748 [INFO] Result on Train Data : {'AUC': 0, 'ACC': 0, 'F1 Score': 0, 'AUPR': 0, 'Loss': 0.4251476288988246}
2024-02-11 18:23:42,192 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-11 18:23:42,193 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-11 18:23:42,195 [INFO] moving model to cpu
2024-02-11 18:23:58,523 [INFO] Epoch: 001, Loss: 0.1788
2024-02-11 18:23:58,524 [INFO] Result on Train Data : {'AUC': 0, 'ACC': 0, 'F1 Score': 0, 'AUPR': 0, 'Loss': 0.17883569210474548}
2024-02-11 18:24:02,550 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-11 18:24:02,551 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
20

# Animate

In [112]:
colors = ['blue', 'red']
cmap = ListedColormap(colors)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10, 10)

def animate(i):
    ax.clear()
    embedding = time_embedds[i]
    ax.scatter(embedding[:, 0], embedding[:, 1], s=10, c=y, cmap=cmap)
    # Set the x and y axis to display a fixed range
    ax.set_xlim([-50, 50])
    ax.set_ylim([-50, 50])

ani = FuncAnimation(fig, animate, frames=len(time_embedds), interval=500, repeat=False)
html_video = ani.to_html5_video()
HTML(html_video)

Output hidden; open in https://colab.research.google.com to view.